# OvenFlow: DSL para descrição de receitas

## Explicando as funções/macros de Scheme utilizadas

A macro `syntax->datum` é necessária para acessar os "dados puros" (datum) dos inputs da nossa macro higiênica quando utilizamos `syntax-case`:

In [9]:
(define-syntax inspect-syntax
  (lambda (x)
    (syntax-case x ()
      ((_ expr)
       (let* ([expr-datum (syntax->datum #'expr)])
         (format #t "Syntax Object capturado: ~s\n" #'expr)
         (format #t "Datum resultante após syntax->datum: ~s\n" expr-datum)
         #'expr)))))

(display "Testando o syntax->datum: ")
(display (inspect-syntax (+ 10 20)))

Syntax Object capturado: #<syntax:unknown file:11:25 (+ 10 20)>
Datum resultante após syntax->datum: (+ 10 20)
Testando o syntax->datum: 30

In [2]:
(define-syntax inspect-syntax
  (lambda (x)
    (syntax-case x ()
      ((_ expr ...)
       (let* ([expr-datum (syntax->datum #'(expr ...))])
         (format #t "Syntax Object capturado: ~s\n"   #'(expr ...))
         (format #t "Datum resultante após syntax->datum: ~s\n" expr-datum))))))

(display "Testando o syntax->datum: ")
(display (inspect-syntax (+ 10 20) (* 2 25)))

Syntax Object capturado: (#<syntax:unknown file:10:25 (+ 10 20)> #<syntax:unknown file:10:35 (* 2 25)>)
Datum resultante após syntax->datum: ((+ 10 20) (* 2 25))
Testando o syntax->datum: #t

Outro ponto importante é a utilização das `association lists`, que podem ser comparadas à estrutura de dicionários/hash tables em outras linguagens, como `Java` ou `C#`. A estrutura delas são como listas onde cada elemento é uma lista composta de 2 posições: primeiro o identificador e por fim o valor. Assim é uma das formas de criar essas listas, forma essa utilizada em nossa DSL:

In [1]:
(let* ((person (acons 'name "Robson" '()))
       (person-with-age (acons 'age 22 person)))
  (format #t "Person: ~a\n" person)
  (format #t "Person with age: ~a\n" person-with-age))

Person: ((name . Robson))
Person with age: ((age . 22) (name . Robson))
#t

Além disso, é necessário a função para consultar os valores dessa lista de acordo com as chaves:

In [4]:
(let* ((person (acons 'name "Robson" '()))
       (person-with-age (acons 'age 22 person)))
  (format #t "Name: ~a\n" (cdr (assv 'name person-with-age)))
  (format #t "Age: ~a\n" (cdr (assv 'age person-with-age))))

Name: Robson
Age: 22
#t

## Funções auxiliadoras

Primeiro, definimos funções para trabalhar com a estrutura de `association list` que escolhemos para tratar as receitas. Elas são compostas por 3 chaves: 
* `name`: o nome da receita.
* `ingredients`: lista de com os ingredientes da receita.
* `steps`: lista com os passos da receita.

In [5]:
; retorna uma lista com duas posições: a primeira é a lista de ingredientes, a segunda é a string com o texto do passo completo.
(define (build-step-result ingredients text-parts)
  (let* ((steps (acons 'steps (string-join (reverse text-parts) " ") '()))
         (with-ingredients (acons 'ingredients (reverse ingredients) steps)))
    with-ingredients))

(define (get-name recipe)
  (cdr (assv 'name recipe)))

(define (get-ingredients recipe)
  (cdr (assv 'ingredients recipe)))

(define (get-steps recipe)
  (cdr (assv 'steps recipe)))

(let  ((recipe-test (build-step-result '("3 ovos" "1 litro de leite") '("1 litro de leite" "com" "3 ovos" "Misturar"))))
  (format #t "Exemplo de receita: ~a\n" recipe-test)
  (format #t "Ingredientes: (~a)\n" (string-join (get-ingredients recipe-test) ", "))
  (format #t "Passos: ~a\n"(get-steps recipe-test)))

Exemplo de receita: ((ingredients 1 litro de leite 3 ovos) (steps . Misturar 3 ovos com 1 litro de leite))
Ingredientes: (1 litro de leite, 3 ovos)
Passos: Misturar 3 ovos com 1 litro de leite
#t

Então, definimos a função `process-step`: ela será responsável por identificar dentro de cada passo quais são os ingredientes e também deve retornar uma string que represente o passo em si.

In [6]:
; verifica se é compatível com (ingredient "alguma string")
(define (is-ingredient? x)
  (and (pair? x) (eq? (car x) 'ingredient)))

; processa cada (step ...)
(define (process-step s)
(let ((elements (cdr s))) ; elimina a palavra step (primeira posição) da lista a ser processada
 (let loop ((rest elements)
            (ingredients '())
            (text-parts '())) ; text-parts é tudo aquilo que foi passado dentro do step que não é um ingredient (texto puro)
   
   (if (null? rest)
       (build-step-result ingredients text-parts)
       (let ((el (car rest)))
         (cond
          
           ; ingrediente — deve ser (ingredient "alguma string")
           ((is-ingredient? el)
            (let ((val (cadr el))) ; elimina a palavra ingredient (primeira posição) da lista a ser processada
              (if (string? val)
                  (loop (cdr rest)
                        (cons val ingredients)
                        (cons val text-parts))
                  (error "Ingrediente deve ser string" el))))
          
           ; texto literal
           ((string? el)
            (loop (cdr rest)
                  ingredients
                  (cons el text-parts)))
          
           ; qualquer outra coisa é inválida
           (else
            (error "Passo inválido" el))))))))

(process-step '(step "Misturar" (ingredient "3 ovos") "," (ingredient "1.5 xícara de açúcar") "e" (ingredient "0.5 xícara de óleo")))

((ingredients "3 ovos" "1.5 xícara de açúcar" "0.5 xícara de óleo") (steps . "Misturar 3 ovos , 1.5 xícara de açúcar e 0.5 xícara de óleo"))

Agora, descrevemos algumas funções auxiliadoras para ajudar na manipulação de listas:
* `take`: retorna apenas os primeiros `n` elementos da lista.
* `drop`: descarta os primeiros `n` elementos da lista.
* `append-item-if-not-present`: insere o `item` na lista, caso não exista.
* `remove-duplicates`: remove todos os itens duplicados da lista, mantendo cada elemento como único.

In [7]:
(define (take lst n)
  (if (or (null? lst) (<= n 0))
      '()
      (cons (car lst) (take (cdr lst) (- n 1)))))

(define (drop lst n)
  (if (or (null? lst) (<= n 0))
      lst
      (drop (cdr lst) (- n 1))))

(define (append-item-if-not-present lst item)
  (if (member item lst)
      lst
      (cons item lst)))

(define (remove-duplicates lst)
  (let loop ((original lst)
             (result '()))
    (if (null? original)
        (reverse result)
        (let ((item (car original)))
          (loop (cdr original) (append-item-if-not-present result item))))))

(format #t "Take: ~a\n" (take '(1 2 3 4 5 6 7 8 9) 3))
(format #t "Drop: ~a\n" (drop '(1 2 3 4 5 6 7 8 9) 3))
(format #t "Append if not present (it is): ~a\n" (append-item-if-not-present '(1 2 3 4 5 6 7 8 9) 3))
(format #t "Append if not present (it's not): ~a\n" (append-item-if-not-present '(1 2 3 4 5 6 7 8 9) 0))
(format #t "Remove duplicates: ~a\n" (remove-duplicates '(1 2 2 3 4 2 5 5 6 1 7 8 9)))

Take: (1 2 3)
Drop: (4 5 6 7 8 9)
Append if not present (it is): (1 2 3 4 5 6 7 8 9)
Append if not present (it's not): (0 1 2 3 4 5 6 7 8 9)
Remove duplicates: (1 2 3 4 5 6 7 8 9)
#t

Falando das modificações que podem ser feitas nas receitas, elas podem se de 3 tipos:
* `add-step-to-start`: adiciona um novo passo como primeiro na receita.
* `add-step-to-end`: adiciona um novo passo como último na receita.
* `add-step-after`: adiciona um novo passo depois do passo de número `index` na receita.

Com isso, precisamos de uma função que faça essa alteração na receita:

In [8]:
(define (insert-new-step action-sym index old-steps new-step-text)
  (case action-sym
       ((add-step-to-start)
        (cons new-step-text old-steps))
       ((add-step-to-end)
        (append old-steps (list new-step-text)))
       ((add-step-after)
        (append (take old-steps index)
                (list new-step-text)
                (drop old-steps index)))
       (else old-steps)))

(format #t "Start: ~a\n" (insert-new-step 'add-step-to-start 2 '(2 3 4 5) 1))
(format #t "End: ~a\n" (insert-new-step 'add-step-to-end 2 '(1 2 3 4) 5))
(format #t "After 2: ~a\n" (insert-new-step 'add-step-after 2 '(1 2 4 5) 3))

Start: (1 2 3 4 5)
End: (1 2 3 4 5)
After 2: (1 2 3 4 5)
#t

E, por conta disso, precisamos de funções que manipulem as modificações e identifiquem as propriedades dela:

In [9]:
(define (find-index-from-modification action-sym action-args)
  (if (eq? action-sym 'add-step-after)
      (car action-args)
      #f))

(format #t "At start: ~a\n" (find-index-from-modification 'add-step-to-start '((step "Adicionar" (ingredient "3 cenouras médias raladas") "à mistura"))))
(format #t "After 2: ~a\n" (find-index-from-modification 'add-step-after '(2 (step "Adicionar" (ingredient "3 cenouras médias raladas") "à mistura"))))

At start: #f
After 2: 2
#t

In [10]:
(define (find-step-from-modification action-sym action-args)
  (if (eq? action-sym 'add-step-after)
      (cadr action-args)
      (car action-args)))

(format #t "At start: ~a\n" (find-step-from-modification 'add-step-to-start '((step "Adicionar" (ingredient "3 cenouras médias raladas") "à mistura"))))
(format #t "After 2: ~a\n" (find-step-from-modification 'add-step-after '(2 (step "Adicionar" (ingredient "3 cenouras médias raladas") "à mistura"))))

At start: (step Adicionar (ingredient 3 cenouras médias raladas) à mistura)
After 2: (step Adicionar (ingredient 3 cenouras médias raladas) à mistura)
#t

## Definindo as macros da DSL

Primeiro, definimos a macro `define-recipe`, que é responsável por criar uma receita base. 

In [11]:
(define-syntax define-recipe
  (lambda (x)
    (syntax-case x ()
      ((_ name step-list ...)
       (let* ((recipe-name (symbol->string (syntax->datum #'name)))
              (raw-steps   (syntax->datum #'(step-list ...))))

         (let* ((processed   (map process-step raw-steps))
                (ingredients (apply append (map get-ingredients processed)))
                (step-texts  (map get-steps processed)))
           (datum->syntax
            x
            `(define ,(syntax->datum #'name)
               '((name . ,recipe-name)
                 (ingredients . ,ingredients)
                 (steps . ,step-texts))))))))))

Depois, definimos a macro `define-modification`, que irá criar as modificações que podem ser aplicadas nas receitas.

In [12]:
(define-syntax define-modification
  (lambda (x)
    (syntax-case x ()
      ((_ mod-name (action . args))
       (let* ((mod-sym     (syntax->datum #'mod-name))
              (action-sym  (syntax->datum #'action))
              (action-args (syntax->datum #'args)))
         (datum->syntax
          x
          `(define ,mod-sym
             (lambda (base-recipe)
               (let* ((action-sym ',action-sym)
                      (action-args ',action-args)
                      ; O índice só é relevante para 'add-step-after'
                      (index (find-index-from-modification action-sym action-args)                       )
                      (step-form (find-step-from-modification action-sym action-args))
                      
                      ; Identifica ingredientes e passo da modificação
                      (processed (process-step step-form))
                      (new-ingredients (get-ingredients processed))
                      (new-step-text  (get-steps processed))
                      
                      ; Identifica ingredientes e passo da receita base
                      (old-ingredients (get-ingredients base-recipe))
                      (old-steps (get-steps base-recipe))
                      
                      ; Faz um "merge" dos ingredientes e passos
                      (new-ingredient-list (append new-ingredients old-ingredients))
                      (new-step-list (insert-new-step action-sym index old-steps new-step-text)))
                 
                       `((name . ,(get-name base-recipe))
                         (ingredients . ,new-ingredient-list)
                         (steps . ,new-step-list)))))))))))



Por último, temos a macro `create-recipe`, que traduzirá as receitas criadas para um texto em Markdown. 

In [13]:
(define-syntax create-recipe
  (syntax-rules ()
    ((_ final-name composition-expr)
     (let ((final-recipe composition-expr))
       (display (string-append "## " final-name "\n\n"))
       
       (display "### Ingredientes\n\n")
       (let ingredients-loop ((ingredients (reverse (remove-duplicates (get-ingredients final-recipe)))))
         (when (not (null? ingredients))
           (display (string-append "* " (car ingredients) "\n"))
           (ingredients-loop (cdr ingredients))))
       
       (display "\n### Modo de Preparo\n\n")
       (let steps-loop ((steps (get-steps final-recipe))
                  (n 1))
         (when (not (null? steps))
           (display (string-append (number->string n) ". " (car steps) "\n"))
           (steps-loop (cdr steps) (+ n 1))))))))




## Testando receitas

In [14]:
(define-recipe bolo
  (step "Misturar" (ingredient "3 ovos") "," (ingredient "1.5 xícara de açúcar") "e" (ingredient "0.5 xícara de óleo"))
  (step "Adicionar" (ingredient "2 xícaras de farinha de trigo") "e misturar bem")
  (step "Assar em forno pré-aquecido a 180 graus por 40 minutos"))

(define-recipe torta
  (step "Bater no liquidificador" (ingredient "3 ovos") "," (ingredient "0.75 xícara de óleo") "," (ingredient "1 xícara de leite") "e" (ingredient "1 colher de chá de sal")))

(define-modification de-cenoura
  (add-step-after 1
   (step "Adicionar" (ingredient "3 cenouras médias raladas") "à mistura e bater novamente")))

(define-modification com-calda-de-chocolate
  (add-step-to-end
   (step "Para a calda, misturar" (ingredient "1 lata de leite condensado") "e" (ingredient "3 colheres de sopa de chocolate em pó") "em fogo baixo até engrossar")))

In [15]:
(display "--- Receita 1: Bolo de Cenoura com Calda ---\n")
(create-recipe "Bolo de Cenoura com Calda de Chocolate"
  (com-calda-de-chocolate (de-cenoura bolo)))

--- Receita 1: Bolo de Cenoura com Calda ---
## Bolo de Cenoura com Calda de Chocolate

### Ingredientes

* 2 xícaras de farinha de trigo
* 0.5 xícara de óleo
* 1.5 xícara de açúcar
* 3 ovos
* 3 cenouras médias raladas
* 3 colheres de sopa de chocolate em pó
* 1 lata de leite condensado

### Modo de Preparo

1. Misturar 3 ovos , 1.5 xícara de açúcar e 0.5 xícara de óleo
2. Adicionar 3 cenouras médias raladas à mistura e bater novamente
3. Adicionar 2 xícaras de farinha de trigo e misturar bem
4. Assar em forno pré-aquecido a 180 graus por 40 minutos
5. Para a calda, misturar 1 lata de leite condensado e 3 colheres de sopa de chocolate em pó em fogo baixo até engrossar


In [16]:
(display "--- Receita 2: Torta de Cenoura ---\n")
(create-recipe "Torta de Cenoura"
  (de-cenoura torta))

--- Receita 2: Torta de Cenoura ---
## Torta de Cenoura

### Ingredientes

* 1 colher de chá de sal
* 1 xícara de leite
* 0.75 xícara de óleo
* 3 ovos
* 3 cenouras médias raladas

### Modo de Preparo

1. Bater no liquidificador 3 ovos , 0.75 xícara de óleo , 1 xícara de leite e 1 colher de chá de sal
2. Adicionar 3 cenouras médias raladas à mistura e bater novamente
